## Visualisation of real estates prices on the map

The result of this project is the visualization of property prices on the map

For this purpose I use:
- pandas library to organize and analyse data containing real estates prices
- geopy, folium and shapely libraries for presentation of real estates prices on the map

### 1. Preparing data

In [ ]:
import pandas as pd

In [ ]:
# source file containing prices of apartments
lokale=pd.read_csv(r'/home/kinga/python/mapki_transakcje_r_wtorny/lokale_wtorny.csv')


In [ ]:
lokale.columns

In [ ]:
# There are some duplicated data here. I have to drop duplicates. Data frame without duplicates is called 'df'

df=lokale.drop_duplicates(subset=['Adres', 'Cena', 'Cena_lokalu', 'Cena_zł/m2',
       'Data_transakcji'])

df.to_csv(r'/home/kinga/python/mapki_transakcje_r_wtorny/df.csv')

In [ ]:
# I divide column 'Adres' into two columns 'Ulica' and 'Numer_mieszkania'
df['ulica'], df['Numer_mieszkania']=df['Adres'].str.split(' m.', 1).str


In [ ]:
df.head()

In [ ]:
# checking how many times the address expressed by street name and house number appears

df.groupby(by=['ulica']).size().sort_values(ascending=False)

In [ ]:
# I create a df in which the name of the street with the house number will appear only once
df_niepowtarzalny_adres=df.drop_duplicates(subset='ulica')

In [ ]:
# checking if the address expressed by street and house number appears only once
df_niepowtarzalny_adres.groupby(by=['ulica']).size().sort_values(ascending=False)

In [ ]:
# Column 'Ulica' contains only street name and house number. I add a new column 'pelen_adres'
# which contains street name,house number and city (Poznań). This columns doesn't contain flat number
df_niepowtarzalny_adres['pelen_adres']=df_niepowtarzalny_adres['ulica']+', Poznań'

In [ ]:
df_niepowtarzalny_adres['pelen_adres'].head()


In [ ]:
# When searching for geographic coordinates, Google understands the address like for example Jeleniogórska 1/3 
# and Jeleniogórska 1/3e as the same address and returns the error that I am asking for the same address too often. 
# Therefore, it is necessary to remove the letter from the building number and then select the base 
# so that each building number appears only once

# function for removing letters at the end of the building number
def nowy_adres(adres):
    if adres[-9].isnumeric()==False:
        return adres[:-9]+adres[-8:]
    else:
        return adres
        


In [ ]:
# checking if function 'nowy_adres' works
nowy_adres('Jeleniogórska 1/3e ,Poznań')

In [ ]:
# I apply function 'nowy_adres'for each row in df_niepowtarzalny_adres, basing on column 'pelen_adres'. 
# Results of this function are shown in a new column 'adres_bez_litery'
df_niepowtarzalny_adres['adres_bez_litery']=df_niepowtarzalny_adres['pelen_adres'].apply(lambda x: nowy_adres(x))

In [ ]:
df_niepowtarzalny_adres.sample(5)

In [ ]:
df_niepowtarzalny_adres['adres_bez_litery']

In [ ]:
df_niepowtarzalny_adres.groupby(by=['adres_bez_litery']).size().sort_values(ascending=False)

In [ ]:
df_niepowtarzalny_adres[df_niepowtarzalny_adres['adres_bez_litery']=='os. Stefana Batorego 82, Poznań']

In [ ]:
# Dropping duplicates - after that each building's number will appear only once
df_niepowtarzalny_adres1=df_niepowtarzalny_adres.drop_duplicates(subset='adres_bez_litery')

In [ ]:
# Function that removes  'ul., al.'  from address adres and replaces 'św.', 'gen.', etc.
def adres_bez_ul (adres):
    return adres.replace('ul. ', '').replace('al. ', '').replace('św.', 'świętego').replace('Józefa','').replace('gen.', '').replace('Stanisława','')
  

In [ ]:
# I apply function adres_bez_ul to each row and replace column 'adres_bez_litery'
df_niepowtarzalny_adres1['adres_bez_litery']=df_niepowtarzalny_adres['adres_bez_litery'].apply(lambda x: adres_bez_ul(x))

In [ ]:
# I check if there is address starting with 'ul.' (there shouldn't be)
df_niepowtarzalny_adres1[df_niepowtarzalny_adres1['adres_bez_litery'].str.startswith('ul')==True].head()

In [ ]:
df_niepowtarzalny_adres1

In [ ]:
# Data frame for which I will establish coordinates
df_niepowtarzalny_adres1.to_csv(r'/home/kinga/python/mapki_transakcje_r_wtorny/baza_do_wspolrzednych.csv')

### 2. Finding geo-coordinates

- to place points on the map, first I need to determine geo coordinates based on the address

In [ ]:
import folium
import time

In [ ]:

def zwrocWspolrzedne(adres):
    """
    Returns geo coordinates based on Nominatim geocoder. If Nominatim fails uses Google geocoder.
    
    Params
    ------
    adres: string
    address to be converted to geo coordinates
    
    Returns
    -------
    tuple - longitude, and latitude
    
    """
    import time
    from geopy.geocoders import GoogleV3, Nominatim
    from geopy import Location
    time.sleep(2) ## Delay execution for a given number of seconds. 
    
    geolocator = Nominatim(user_agent="my-application", timeout=3)
    try:
        wspolrzedne = geolocator.geocode(adres)[1]
    except TypeError:
        wspolrzedne=adres
    print('Nominatim: ',wspolrzedne)
    
    if isinstance(wspolrzedne,tuple)==False:
        geolocator = GoogleV3(user_agent="my-application", timeout=3)
        wspolrzedne = geolocator.geocode(adres)[1]
        print('Google: ',wspolrzedne)
        
    return wspolrzedne

In [ ]:
zwrocWspolrzedne('świętego. Jacka 3, Poznań')

In [ ]:
# Using 'zwrocWspolrzedne' function, I find geo coordinates of each address in 'df_niepowtarzalny_adres1'
df_niepowtarzalny_adres1['wspolrzedne']=df_niepowtarzalny_adres1.adres_bez_litery.apply(lambda x: zwrocWspolrzedne(x)).head()


### 3. Completing  geo coordinates in the' df '  with coordinates from  'df_niepowtarzalny_adres1'

In [ ]:
## data frame with all transactions : 'df'
## data frame containing geo coordinates: 'df_niepowtarzalny_adres1'

In [ ]:
for row in df_niepowtarzalny_adres1.iterrows():
    df.loc[(df.ulica.str.contains(row[1]['adres_bez_litery']), 'wspolrzedne')]=row[1]['wspolrzedne']

In [ ]:
## data frame with all geo coordinates
df.to_csv(r'/home/kinga/python/mapki_transakcje_r_wtorny/baza_cala.xlsx')

### 4. Dividing geo coordinates into two columns: longitude and lattitude

In [ ]:
baza_cala=pd.read_excel(r'/home/kinga/python/mapki_transakcje_r_wtorny/baza_cala.xlsx')

In [ ]:
## I remove brackets from column 'wspolrzedne'
def wspolrzedne_bez_nawiasu (wspolrzedne):
    return wspolrzedne.replace('(','').replace(')','')

In [ ]:
baza_cala['wspolrzedne']=baza_cala.wspolrzedne.apply(lambda x: wspolrzedne_bez_nawiasu (x))

In [ ]:
## Dividing geo coordinates into two columns: longitude and lattitude

In [ ]:
baza_cala['dlugosc'], baza_cala['szerokosc']=baza_cala['wspolrzedne'].str.split(',').str

In [ ]:
baza_cala.loc[:,'szerokosc']= pd.to_numeric(baza_cala['szerokosc'])

In [ ]:
## data frame with all geo coordinates divided into two columns: longitute and lattitiude
baza_cala.to_excel(r'/home/kinga/python/mapki_transakcje_r_wtorny/wszystkie_wspolrzedne.xlsx')

### 5. Price analysis, removing outliers

In [ ]:
import pandas as pd
baza_cala=pd.read_excel(r'/home/kinga/python/mapki_transakcje_r_wtorny/wszystkie_wspolrzedne.xlsx')

In [ ]:
obreb=baza_cala.groupby(by=['Obręb'], as_index=True)['Cena_zł/m2']

In [ ]:
obreb

In [ ]:
# I build fata frame 'df_conc' which contains paramiters like min, max, std etc.
df_conc = pd.concat([                     obreb.apply(lambda x: x.std()),\
                     obreb.apply(lambda x: x.min()),\
                    obreb.apply(lambda x: x.max()),\
                    obreb.apply(lambda x: x.mean()),\
                    obreb.apply(lambda x: (x.mean()+x.std()*1.96)),\
                    obreb.apply(lambda x: (x.mean()-x.std()*1.96))                
                    ],axis=1)
nazwy_kolumn = ['std_zł/m2','min_zł/m2', 'max_zł/m2', 'mean_zł/m2', 'toprange', 'botrange']
df_conc.columns=nazwy_kolumn
df_conc

In [ ]:
df_conc.head()

In [ ]:
# I remove from 'baza_cala' transactions that are <botrange and >toprange
df_odfiltrowane = pd.DataFrame()

#for element df.groupby returns tuple: nazwa_grupy, dataframe of tis group
for nazwa_grupy,dataframe in baza_cala.groupby(by=['Obręb']):

    toprange=df_conc.loc[nazwa_grupy,'toprange']
    botrange=df_conc.loc[nazwa_grupy,'botrange']
#      print(nazwa_grupy,',toprange: ',toprange), print(nazwa_grupy,', botrange: ',botrange)
    df_filtr_grupa=dataframe[(dataframe['Cena_zł/m2']<toprange)&(dataframe['Cena_zł/m2']>botrange)]
    df_odfiltrowane = pd.concat([df_odfiltrowane,df_filtr_grupa])

In [ ]:
# data frame without outliers
df_odfiltrowane[['Obręb','Cena_zł/m2']].groupby(by='Obręb').describe().head()

In [ ]:
df_odfiltrowane.to_excel(r'/home/kinga/python/mapki_transakcje_r_wtorny/baza_bez_outliers.xlsx')

### 6.  Map 1

- this map contains points that show the prices of apartments in specific locations
- in the right upper corner there is a menu in which you can choose the area of flats to be visible on the map

In [4]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import html


In [6]:
# There is about 2500 transactions in my data base and when I put them all on the map, it gets unreadable. 
# That's why I'll show only transactions with price over 7000 zł/m2. 
# despite of this, it is possible to choose different criteria to show other transactions on the map
baza_bez_outliers=pd.read_excel(r'/home/kinga/python/Mapki_transakcje_r_wtorny/baza_bez_outliers.xlsx')
baza_do_mapki=baza_bez_outliers.loc[baza_bez_outliers['Cena_zł/m2']>=7000] # conatins only transations with price over 7000zl/m2

In [7]:

baza_do_mapki.groupby(by=['Obręb']).size().sort_values(ascending=False) # shows how many transactions there is in each Obręb

Obręb
KOMANDORIA    41
POZNAŃ        32
ŁAZARZ        20
JEŻYCE        16
WILDA         12
WINIARY        8
RATAJE         6
PIĄTKOWO       6
ŁAWICA         5
GŁÓWNA         2
GOLĘCIN        2
ŚRÓDKA         1
NARAMOWICE     1
JUNIKOWO       1
dtype: int64

In [8]:

map1=folium.Map(location=[52.391097, 16.846651], zoom_start=11)
marker_cluster1=MarkerCluster(name='Powierzchnia <=40 m2').add_to(map1)
marker_cluster2=MarkerCluster(name='Powierzchnia 40 <=70 m2 ').add_to(map1)
marker_cluster3=MarkerCluster(name='Powierzchnia >70 m2').add_to(map1)

for row in baza_do_mapki.iterrows():
    if row[1]['powierzchnia_lokalu']<=40:
        folium.Marker((row[1]['dlugosc'],row[1]['szerokosc']),\
        popup=html.escape('cena: ' + str(row[1]['Cena_zł/m2']) +' zł/m2, ' + 'powierzchnia: ' +str(row[1]['powierzchnia_lokalu']) +' m2'),\
        icon=folium.Icon(color='black')).add_to(marker_cluster1)
    elif row[1]['powierzchnia_lokalu']>40 and row[1]['powierzchnia_lokalu']<=70:
        folium.Marker((row[1]['dlugosc'],row[1]['szerokosc']),\
        popup=html.escape('cena: ' + str(row[1]['Cena_zł/m2']) +' zł/m2, ' + 'powierzchnia: '+ str(row[1]['powierzchnia_lokalu']) +' m2'),\
        icon=folium.Icon(color='black')).add_to(marker_cluster2)
    else:
        folium.Marker((row[1]['dlugosc'],row[1]['szerokosc']),\
        popup=html.escape('cena: ' + str(row[1]['Cena_zł/m2']) +' zł/m2, ' + 'powierzchnia: '+ str(row[1]['powierzchnia_lokalu']) +' m2'),\
        icon=folium.Icon(color='black')).add_to(marker_cluster3)
        
        
# map1.add_child(folium.ClickForMarker(popup='Waypoint'))

folium.LayerControl().add_to(map1) 
map1

In [11]:
map1.save('mapa.html')

In [12]:
from IPython.display import IFrame

IFrame(src='mapa.html', width=900, height=600)

In [ ]:
%%html
<a href="mapa.html">link</a>

### 7. Division into districts

In [ ]:
from shapely.geometry import Point, Polygon, MultiPoint

In [ ]:
# using the geo coordinates I draw a polygon that marks the boundaries of districts
import json
jezyce=None
with open('dzielnice_poznan.json') as json_data:
    d = json.load(json_data)
    print (d['features'][0]['geometry']['coordinates'][0])# prints geo coordinates of first district
    print (d['features'][0]['properties']['name'])# prints name of first district
    jezyce=MultiPoint(d['features'][0]['geometry']['coordinates'][0]).convex_hull # places geo coordinates in the right order
    jezyce1=Polygon(d['features'][0]['geometry']['coordinates'][0])

In [ ]:
jezyce1

In [ ]:
#I create a series in which polygons will be assigned to districts.
# Then I will check in which district(polygon) addresses from data frame are
sr_dzielnice = pd.Series()

with open('dzielnice_poznan.json') as json_data:
    d = json.load(json_data)
    for x in range(len(d['features'])): ## len features, because each district is separate feature and has to ask for all districts
        wielokat = Polygon(d['features'][x]['geometry']['coordinates'][0]) # geo coordinates each district
        sr_dzielnice=pd.concat([sr_dzielnice,pd.Series(wielokat,index=[d['features'][x]['properties']['name']])]) # index=districts's names

In [ ]:
sr_dzielnice.head()

In [ ]:
# the function checks the district in which the individual address is
def dzielnica (szerokosc,dlugosc):
    return ','.join(sr_dzielnice[sr_dzielnice.apply(lambda polygon: polygon.contains(Point(szerokosc,dlugosc)))==True].index.values) 

In [ ]:
dzielnica(16.982495, 52.3936962)

In [ ]:
# check distric od each address from data frame 'baza_bez_outliers'
baza_bez_outliers['dzielnica']=baza_bez_outliers.apply(lambda x: dzielnica(x['szerokosc'], x['dlugosc']) , axis=1)



In [ ]:
# data frame with districts
baza_bez_outliers.to_excel(r'/home/kinga/python/mapki_transakcje_r_wtorny/baza_bez_outliers_dzielnice.xlsx')

In [ ]:
baza_bez_outliers_dzielnice=pd.read_excel(r'/home/kinga/python/baza_bez_outliers_dzielnice.xlsx')

In [ ]:
dzielnica=baza_bez_outliers_dzielnice.groupby('dzielnica')['Cena_zł/m2']

In [ ]:
#I build fata frame 'df_conc_dzielnice' which contains paramiters like min, max,mean etc.
dzielnica=baza_bez_outliers_dzielnice.groupby('dzielnica')['Cena_zł/m2']
df_conc_dzielnice=pd.concat([dzielnica.apply(lambda x: x.min()),\
                           dzielnica.apply(lambda x: x.max()),\
                           dzielnica.apply(lambda x: x.mean()),\
                           baza_bez_outliers_dzielnice.groupby(by=['dzielnica']).size()],\
                           axis=1, ignore_index=True)
columns=['zł/m2 min', 'zł/m2 max', 'zł/m2 sr', 'liczba transakcji']
df_conc_dzielnice.columns=columns

In [ ]:
df_conc_dzielnice.reset_index(inplace=True) # I change 'dzielnica' from index into column

In [ ]:
df_conc_dzielnice

### 8. Map 2 -choropleth

- on this map I present what prices appear in the area of a given district. The map presents minimal prices, but depending needs, it is also possible to present maximum or average prices
- ot the choropleth map areas are shaded in proportion to the measurement of the statistical variable being displayed on the map

In [ ]:
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
m = folium.Map(location=[52.391097, 16.846651], zoom_start=10.5)
poznan_dzielnice = 'dzielnice_poznan.json'

m.choropleth(
    geo_data=poznan_dzielnice,
    data=df_conc_dzielnice,
    columns=['dzielnica', 'zł/m2 min'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    legend_name='minimalna cena zł/m2')


folium.LayerControl().add_to(m)

m.save('GeoJSON_and_choropleth_9.html')

m

In [ ]:
from IPython.display import IFrame

IFrame(src='GeoJSON_and_choropleth_9.html', width=900, height=600)

In [ ]:
%%html
<a href="GeoJSON_and_choropleth_9.html">link</a>